In [1]:
from transformer import Transformer # this is the transformer.py file
import torch
import numpy as np

In [2]:
english_file = '/content/drive/MyDrive/Colab Notebooks/en-ta/train.en'
tamil_file = '/content/drive/MyDrive/Colab Notebooks/en-ta/train.ta'

In [3]:
START_TOKEN = '<START>'
PADDING_TOKEN = '<PADDING>'
END_TOKEN = '<END>'

tamil_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                      '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', '?', 'ˌ',
                     "ஃ", "அ", "ஆ", "இ", "ஈ", "உ", "ஊ", "எ", "ஏ", "ஐ", "ஒ", "ஓ", "ஔ",
                      "க", "ங", "ச", "ஜ", "ஞ", "ட", "ண",
                      "த", "ந", "ன", "ப", "ம", "ய", "ர",
                      "ற", "ல", "ள", "ழ", "வ", "ஶ", "ஷ", "ஸ", "ஹ",
                      "ா", "ி", "ீ", "ு", "ூ", "ெ", "ே", "ை", "ொ", "ோ", "ௌ", "்", "ௗ",
                      "௦", "௧", "௨", "௫", "௬", "௲", "௳", PADDING_TOKEN, END_TOKEN]

english_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@',
                        '[', '\\', ']', '^', '_', '`',
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                        'y', 'z',
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

In [4]:
index_to_tamil= {k:v for k,v in enumerate(tamil_vocabulary)}
tamil_to_index = {v:k for k,v in enumerate(tamil_vocabulary)}
index_to_english = {k:v for k,v in enumerate(english_vocabulary)}
english_to_index = {v:k for k,v in enumerate(english_vocabulary)}

In [5]:
from google.colab import drive
drive.mount('/content/drive')

with open(english_file, 'r') as file:
    english_sentences = file.readlines()
with open(tamil_file, 'r') as file:
    tamil_sentences = file.readlines()

Mounted at /content/drive


In [6]:
# Limit Number of sentences
TOTAL_SENTENCES = 200000
english_sentences = english_sentences[:TOTAL_SENTENCES]
tamil_sentences = tamil_sentences[:TOTAL_SENTENCES]
english_sentences = [sentence.rstrip('\n').lower() for sentence in english_sentences]
tamil_sentences = [sentence.rstrip('\n') for sentence in tamil_sentences]

In [7]:
english_sentences[:10]

['is this true ? ” asked jenny , a pretty teenager .',
 '“ and if it is , why do we humans have only a simple stomach when we are supposed to be much higher than the cow on the evolution scale ? ” “ well , first of all it must be said that , strictly speaking , the cow has only one stomach , but it is divided into four compartments . those animals that eat mostly grass and hay and chew the cud are called ruminants .',
 'let me draw you a picture on this old envelope . . . the first compartment is the rumen , which has an average capacity of some 50 gallons , and it occupies about 80 percent of the stomach space in the cow . the abomasum , actually the fourth compartment , is the true digestive stomach , similar to our human stomach with its gastric juices , including hydrochloric acid . the grass , mixed well with saliva , goes into the rumen and reticulum where it soaks in water and is subjected to bacterial action . later on it is regurgitated , or brought back into the mouth , to be

In [8]:
tamil_sentences[:10]

['நீங்கள் என்றும் வாழலாம் !',
 'நீங்கள் பூமியில் பரதீஸில் என்றும் வாழலாம் என்ற புதிய புத்தகத்தைக் குறித்து போற்றுதலுள்ள ஒரு வாசகர் , அது “ ஜீவனைத் தொடர்ந்து காக்கும் சத்தியங்களால் ” நிறைந்திருக்கிறது என்று எழுதினார் .',
 'ஒரு முழுநேர கிறிஸ்தவ பிரசங்கியாக மற்ற குடும்பங்களுடன் அநேக வேதப்படிப்புகளை நடத்தும் அவள் சொன்னாள் : “ இந்தப் புதிய புத்தகம்தான் தேவையைப் பூர்த்தி செய்கிறது !',
 'இது குழந்தைகள் சத்தியத்தைக் கிரகித்துக்கொள்வதற்குப் போதுமான அளவு எளிதாகவும் பெற்றோர் தங்களுடைய தீர்மானங்களைச் செய்வதற்குப் போதுமான அளவு புத்திக்கூர்மையுள்ளதாகவும் இருக்கிறது .',
 'இது சுருக்கமாகவும் நேரடியாகவும் கையாளுகிறது .',
 'இது காரியங்களைச் சுற்றிவளைத்து பேசுவதில்லை !',
 'இது ‘ சத்தியத்தை ’ அது இருக்கும் வண்ணமாகவே தெரிவிக்கிறது .',
 'நேர்மை இருதயமுள்ளவர்களின் இருதயத்தைச் சென்றெட்டுவதற்கான தேவையான எல்லாச் சிறந்த குறிப்புகளாலும் நிறைந்திருக்கிறது .',
 'இந்தப் புதிய வெளியீடு சங்கத்தின் எல்லாப் பிரசுரங்களையும் ஒரு சிறு புத்தகமாக அமைத்திருப்பதுபோல் இருக்கிறது . ”',
 'வேதப்படிப்பிற்கென்று வடிவமைக்கப்பட்ட சுலப

In [9]:
import numpy as np
PERCENTILE = 97
print( f"{PERCENTILE}th percentile length Tamil: {np.percentile([len(x) for x in tamil_sentences], PERCENTILE)}" )
print( f"{PERCENTILE}th percentile length English: {np.percentile([len(x) for x in english_sentences], PERCENTILE)}" )

97th percentile length Tamil: 210.0
97th percentile length English: 261.0


In [10]:
!pip install torch
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
class SentenceEmbedding(nn.Module):
    def __init__(self, vocab, start_token, end_token):
        super(SentenceEmbedding, self).__init__()
        self.vocab = vocab
        self.start_token = start_token
        self.end_token = end_token
    def forward(self, sentences):
        pass



    def tokenize(self, sentence):
        tokens = [self.vocab[self.start_token]]
        tokens.extend([self.vocab[token] for token in sentence if token in self.vocab])
        tokens.append(self.vocab[self.end_token])
        return tokens

    def batch_tokenize(self, batch_sentences, start_token=True, end_token=True):
        return pad_sequence([torch.tensor(self.tokenize(sentence)) for sentence in batch_sentences], batch_first=True)
max_sequence_length = 200
min_sequence_length = 3

def is_valid_tokens(sentence , vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length , min_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) and len(list(sentence)) > min_sequence_length + 1  # need to re-add the eos and bos

valid_sentence_indicies = []
for index in range(len(english_sentences[:1000000])):
    tamil_sentence, english_sentence = tamil_sentences[index], english_sentences[index]
    if is_valid_length(english_sentence, max_sequence_length , min_sequence_length)  \
      and is_valid_length(tamil_sentence, max_sequence_length ,min_sequence_length) \
      and is_valid_tokens(tamil_sentence, tamil_vocabulary) \
      and is_valid_tokens(english_sentence , english_vocabulary):
        valid_sentence_indicies.append(index)

print(f"Number of sentences: {len(tamil_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")

Number of sentences: 200000
Number of valid sentences: 90128


In [11]:
tamil_sentences = [tamil_sentences[i] for i in valid_sentence_indicies]
english_sentences = [english_sentences[i] for i in valid_sentence_indicies]

In [12]:
tamil_sentences[24:27]

['குறைபாடுகளை பொறுத்துக் கொள்ளக்கூடுமா ?',
 'சில ஆசிரியர்கள் ஏன் இந்த முறையில் நடந்து கொள்ளுகிறார்கள் .',
 'ஒருவேளை அது அவ்வாறு தோன்றக்கூடும் .']

In [13]:
import torch

d_model = 512
batch_size = 30
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 1
max_sequence_length = 200
ta_vocab_size = len(tamil_vocabulary)

transformer = Transformer(d_model,
                          ffn_hidden,
                          num_heads,
                          drop_prob,
                          num_layers,
                          max_sequence_length,
                          ta_vocab_size,
                          english_to_index,
                          tamil_to_index,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)

In [14]:
transformer

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(71, 512)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (linear_layer): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionwiseFeedForward(
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNormalization()
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Decoder(
    (sentence_embedding):

In [15]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):

    def __init__(self, english_sentences, tamil_sentences):
        self.english_sentences = english_sentences
        self.tamil_sentences = tamil_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.tamil_sentences[idx]

In [16]:
dataset = TextDataset(english_sentences, tamil_sentences)

In [17]:
len(dataset)

90128

In [18]:
dataset[1]

('the book discusses practically every controversial bible teaching , gathering the evidence together in such a concise and understandable way that the answer becomes clear to the reader .',
 'அத்தாட்சிகளை அத்தனை சுருக்கமாகவும் புரிந்துக் கொள்ளத்தக்க வகையிலும் ஒன்றாகச் சேர்த்துத் தருவதால் வாசகருக்குப் பதில் தெளிவாக தெரிந்துவிடுகிறது .')

In [19]:
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

In [20]:
for batch_num, batch in enumerate(iterator):
    print(batch)
    if batch_num > 3:
        break

[('we feel that you , too , will be so excited about the message in this book that you will be delighted to share things you have read with others .', 'the book discusses practically every controversial bible teaching , gathering the evidence together in such a concise and understandable way that the answer becomes clear to the reader .', 'only $ 2.50 . please send , postpaid , you can live forever in paradise on earth . i enclose $ 2.50 ( u.s . ) .', '( for price in other countries , please contact local watch tower society office . )', 'listening to the great teacher is a unique book for parents and children . the stories are designed in a way that encourages you to ask your child questions .', 'this , in turn , encourages your child to become more involved and to learn to think . equally important , it helps you to hear how your child really feels about things .', 'please send , postpaid , the 192 - page , hardcover book listening to the great teacher .', 'i enclose $ 1 ( u.s . ) .'

In [21]:
from torch import nn

criterian = nn.CrossEntropyLoss(ignore_index=tamil_to_index[PADDING_TOKEN],
                                reduction='none')

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [22]:
NEG_INFTY = -1e9

def create_masks(eng_batch, ta_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
      eng_sentence_length, ta_sentence_length = len(eng_batch[idx]), len(ta_batch[idx])
      eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
      ta_chars_to_padding_mask = np.arange(ta_sentence_length + 1, max_sequence_length)
      encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
      encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[idx, :, ta_chars_to_padding_mask] = True
      decoder_padding_mask_self_attention[idx, ta_chars_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
      decoder_padding_mask_cross_attention[idx, ta_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

In [23]:

transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        eng_batch, ta_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, ta_batch)
        optim.zero_grad()
        ta_predictions = transformer(eng_batch,
                                     ta_batch,
                                     encoder_self_attention_mask.to(device),
                                     decoder_self_attention_mask.to(device),
                                     decoder_cross_attention_mask.to(device),
                                     enc_start_token=False,
                                     enc_end_token=False,
                                     dec_start_token=True,
                                     dec_end_token=True)
        # Use SentenceEmbedding for batch tokenization
        sentence_embedding = SentenceEmbedding(tamil_to_index, START_TOKEN, END_TOKEN)
        labels = sentence_embedding.batch_tokenize(ta_batch)
        labels = transformer.decoder.sentence_embedding.batch_tokenize(ta_batch, start_token=False, end_token=True)
        loss = criterian(
            ta_predictions.view(-1, ta_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == tamil_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        #train_losses.append(loss.item())
        if batch_num % 100 == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"Tamil Translation: {ta_batch[0]}")
            ta_sentence_predicted = torch.argmax(ta_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in ta_sentence_predicted:
              if idx == tamil_to_index[END_TOKEN]:
                break
              predicted_sentence += index_to_tamil[idx.item()]
            print(f"Tamil Prediction: {predicted_sentence}")


            transformer.eval()
            ta_sentence = ("",)
            eng_sentence = ("should we go to the mall?",)
            for word_counter in range(max_sequence_length):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, ta_sentence)
                predictions = transformer(eng_sentence,
                                          ta_sentence,
                                          encoder_self_attention_mask.to(device),
                                          decoder_self_attention_mask.to(device),
                                          decoder_cross_attention_mask.to(device),
                                          enc_start_token=False,
                                          enc_end_token=False,
                                          dec_start_token=True,
                                          dec_end_token=False)
                next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_tamil[next_token_index]
                ta_sentence = (ta_sentence[0] + next_token, )
                if next_token == END_TOKEN:
                  break

            # Printing and evaluation code here
            print(f"Evaluation translation (should we go to the mall?) : {ta_sentence}")
            print("-------------------------------------------")

Epoch 0
Iteration 0 : 5.370614528656006
English: we feel that you , too , will be so excited about the message in this book that you will be delighted to share things you have read with others .
Tamil Translation: இந்தப் புத்தகம் உண்மையில் வாதத்துக்கு இடமளிக்கும் ஒவ்வொரு பைபிள் போதகத்தையும் கலந்தாராய்கிறது .
Tamil Prediction: =௳௳௳௳)வஶறற=௳==௳௳=௳௳௳௳====எ==௳===௳௳௳௳=ஹ=ற993௲77௲ஹ௳௳௳௳௳௳௲௲௳ஹ௳8=898௲=௳7௳௳௳௳=7=ஐ௳௳்௳௲்=ஹ்்=௳==='====1உ3ஐ=௲௲௳1111௲ஶ௳௳எவஞ௳ஞ்்௳77இவ7௲த௲73௲33337௳வ௳௫௫33௳'௳்்௲௳௲ஹஹஹஹ3வவஶஹஹ3ஹஹஶஹ$ஹஜஹஜஹஜஜஐ௲ஹஹஜ்ஐஐ்ஶஹஶஶஹவஜஹஜபப=ஐ7=௳௳
Evaluation translation (should we go to the mall?) : ('்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்தததத்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்்த்ஜஜ்்்்்்்்்்்்்்்்்்்்்த்்்்்்்்',)
-------------------------------------------
Iteration 100 : 3.2175042629241943
English: pupil violence grows
Tamil Translation: மாணாக்கர் ஒருவரையொருவர் கத்தியால் குத்திக்கொள்ளுகிறார்கள் , சிறுவர்களைக் கட்டட மாடி 

In [24]:
transformer.eval()
def translate(eng_sentence):
  eng_sentence = (eng_sentence,)
  ta_sentence = ("",)
  for word_counter in range(max_sequence_length):
    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, ta_sentence)
    predictions = transformer(eng_sentence,
                              ta_sentence,
                              encoder_self_attention_mask.to(device),
                              decoder_self_attention_mask.to(device),
                              decoder_cross_attention_mask.to(device),
                              enc_start_token=False,
                              enc_end_token=False,
                              dec_start_token=True,
                              dec_end_token=False)
    next_token_prob_distribution = predictions[0][word_counter]
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = index_to_tamil[next_token_index]
    ta_sentence = (ta_sentence[0] + next_token, )
    if next_token == END_TOKEN:
      break
  return ta_sentence[0]

In [25]:
translation = translate("what should we do when the day starts?")
print(translation)
#நாள் தொடங்கும் போது நாம் என்ன செய்ய வேண்டும்?

அது எப்படி செய்யலாம் ?<END>


In [26]:
translation = translate("how is this the truth?")
print(translation)
#இது எப்படி உண்மை?

அது எப்படி ?<END>


In [27]:
translation = translate("the world is a large place with different people")
print(translation)
#உலகம் வெவ்வேறு மனிதர்களைக் கொண்ட ஒரு பெரிய இடம்

பிரச்சினை பார்த்து<END>


In [29]:
translation = translate("my name is dikashma")
print(translation)
#என் பெயர் திகஷ்மா ஸ்ரீ

பொருளடக்கம்<END>


In [30]:
translation = translate("i cannot stand this smell")
print(translation)
#இந்த வாசனையை என்னால் தாங்க முடியவில்லை

பிரிட்டன் பிள்ளைகள்<END>


In [31]:
translation = translate("vegetables are the best")
print(translation)
#காய்கறிகள் சிறந்தவை

பிரச்சினை செய்ய முடியும் பிரச்சினைகள்<END>


In [32]:
translation = translate("why care about this?")
print(translation)
#இதைப் பற்றி ஏன் கவலைப்பட வேண்டும்?

அது எப்படி ?<END>


In [33]:
translation = translate("good morning,how do you do?")
print(translation)


அது எப்படி ?<END>


In [34]:
translation = translate("i am here")
print(translation)
#நான் இங்கே இருக்கிறேன்

பிரான்<END>


In [35]:
translation = translate("what is that?")
print(translation)

அது என்ன ?<END>


In [ ]:
translation = translate("")
print(translation)

In [ ]:
translation = translate("")
print(translation)

In [ ]:
translation = translate("")
print(translation)

In [ ]:
translation = translate("")
print(translation)

In [ ]:
translation = translate("")
print(translation)